<a href="https://colab.research.google.com/github/mahikkaaa/Carbon-Cure/blob/main/IOT_Device.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import serial
import time
import pickle  # Import the pickle module for loading the machine learning model
from gpiozero import PWMLED
from gps import GPS
from bluetooth import BluetoothSerial
from acd import ACD10

In [ ]:
# Load the machine learning model
with open('path/to/your/model.pkl', 'rb') as model_file:
    machine_learning_model = pickle.load(model_file)


In [ ]:
# Initialize CO2 sensor (ACD10)
co2_sensor = ACD10()

# Initialize Bluetooth serial connection
bluetooth_serial = BluetoothSerial(port='/dev/rfcomm0', baudrate=9600)

# Initialize GPS module
gps = GPS(port='/dev/ttyS0', baudrate=9600)

# Initialize battery status LED
battery_led = PWMLED(17)  # GPIO Pin 17


In [ ]:
def measure_and_send_data():
    # Measure CO2 level
    co2_level = co2_sensor.measure_co2()

    # Get GPS coordinates
    latitude, longitude = gps.get_coordinates()

    # Prepare data
    data = f"CO2 Level: {co2_level} ppm | Latitude: {latitude} | Longitude: {longitude}"

    # Send data via Bluetooth
    bluetooth_serial.write(data.encode())

    # Print data for local debugging
    print(data)

    # Make prediction using the machine learning model
    prediction_input = [co2_level, latitude, longitude]
    prediction = machine_learning_model.predict([prediction_input])

    # Print the machine learning prediction
    print(f"Machine Learning Prediction: {prediction[0]}")


In [ ]:
def check_battery_status():
    # This is a placeholder function; you may need to implement a proper battery status check based on your battery specifications
    battery_status = get_battery_status()

    # Adjust battery LED brightness based on battery status
    battery_led.value = battery_status / 100.0


In [ ]:
# Main loop
while True:
    try:
        measure_and_send_data()
        check_battery_status()
        time.sleep(60)  # Adjust the sleep interval based on your requirements
    except KeyboardInterrupt:
        break
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
# Cleanup
co2_sensor.cleanup()
bluetooth_serial.close()
gps.close()
